In [1]:
'''
#FAFB LNd6: 720575940634984800-->left, 720575940627933336-->right
#FAFB 5LNv: 720575940625254636-->left, 720575940619074049-->right
#VP5 + SEZ adPN 720575940623617973 -->left, 720575940620445062 -->right
'''

'\n#FAFB LNd6: 720575940634984800-->left, 720575940627933336-->right\n#FAFB 5LNv: 720575940625254636-->left, 720575940619074049-->right\n#VP5 + SEZ adPN 720575940623617973 -->left, 720575940620445062 -->right\n'

In [2]:
from fafbseg import flywire
import navis
import pandas as pd
from collections import deque
from tqdm import tqdm
token = "your token"
flywire.set_chunkedgraph_secret(token, overwrite = True)
flywire.set_default_dataset('production')

Token succesfully stored.
Default dataset set to "production".


In [3]:
def find_bidirectional_paths(source, target, max_steps=6, weight_threshold=5, min_steps=2):
    """
    Find candidate paths from source to target using bidirectional search.
    
    The search expands from the source (forward) by following outgoing edges
    and from the target (backward) by following incoming edges.
    
    Parameters:
      - source: Source neuron ID.
      - target: Target neuron ID.
      - max_steps: Maximum total number of edges allowed (forward_depth + backward_depth).
      - weight_threshold: Only consider connections with weight >= this threshold.
      - min_steps: Minimum number of edges for a valid path.
    
    Returns:
      A list of candidate paths. Each candidate is a tuple (path, weights) where:
         * path: list of neuron IDs from source to target.
         * weights: list of synaptic weights for each edge.
    """
    # Initialize forward and backward candidate lists.
    # Forward candidates: paths starting at source.
    # Backward candidates: paths starting at target.
    forward = [([source], [])]
    backward = [([target], [])]  # Note: these paths are built from target backward.
    
    forward_depth = 0
    backward_depth = 0
    
    # The search stops when total depth (forward + backward) reaches max_steps.
    while (forward or backward) and (forward_depth + backward_depth < max_steps):
        # First, check for intersection between frontiers.
        forward_frontier = { path[-1] for path, _ in forward }
        backward_frontier = { path[-1] for path, _ in backward }
        intersect = forward_frontier.intersection(backward_frontier)
        if intersect:
            merged_paths = []
            for node in intersect:
                # For every candidate in forward ending with the intersection node...
                for fpath, fweights in [cand for cand in forward if cand[0][-1] == node]:
                    # ...and every candidate in backward ending with that node.
                    for bpath, bweights in [cand for cand in backward if cand[0][-1] == node]:
                        # Merge: fpath + reversed(bpath[:-1]) (avoid duplicating the intersection node)
                        merged_path = fpath + list(reversed(bpath[:-1]))
                        merged_weights = fweights + list(reversed(bweights))
                        if (len(merged_path) - 1) >= min_steps:
                            merged_paths.append((merged_path, merged_weights))
            if merged_paths:
                return merged_paths

        # Decide which side to expand next based on the number of candidates.
        if len(forward) <= len(backward):
            # Expand the forward side.
            new_forward = []
            frontier_nodes = { path[-1] for path, _ in forward }
            # Batch query: get all connectivity for the current forward frontier.
            df = flywire.get_connectivity(list(frontier_nodes))
            # Keep only rows where the 'pre' is in the frontier.
            df = df[df['pre'].isin(frontier_nodes)]
            df = df[df['weight'] >= weight_threshold]
            for path, weights in forward:
                current = path[-1]
                sub_df = df[df['pre'] == current]
                for row in sub_df.itertuples(index=False):
                    post = row.post
                    w = row.weight
                    # Avoid cycles within the forward search.
                    if post in path:
                        continue
                    new_forward.append((path + [post], weights + [w]))
            forward = new_forward
            forward_depth += 1
        else:
            # Expand the backward side.
            new_backward = []
            frontier_nodes = { path[-1] for path, _ in backward }
            # For backward, we need to get connectivity where the candidate is in the 'post' column.
            df = flywire.get_connectivity(list(frontier_nodes))
            df = df[df['post'].isin(frontier_nodes)]
            df = df[df['weight'] >= weight_threshold]
            for path, weights in backward:
                current = path[-1]
                sub_df = df[df['post'] == current]
                for row in sub_df.itertuples(index=False):
                    pre = row.pre
                    w = row.weight
                    if pre in path:
                        continue
                    new_backward.append((path + [pre], weights + [w]))
            backward = new_backward
            backward_depth += 1
    return []  # No path found within the allowed maximum steps.

In [4]:
def automate_bidirectional_paths(sources, target, max_steps=6, weight_threshold=5, min_steps=2):
    """
    For each source neuron in a list, run bidirectional search to find candidate pathways to the target.
    
    Returns:
      A list of dictionaries. Each dictionary represents one found path with the following keys:
         - 'source': the source neuron ID.
         - 'target': the target neuron ID.
         - 'path': the raw list (as a string) of neuron IDs from source to target.
         - 'formatted_path': a formatted string representation of the pathway.
         - 'weights': the list (as a string) of connection weights.
         - 'steps': the number of edges in the path.
    """
    results = []
    for source in tqdm(sources, desc="Processing sources"):
        candidate_paths = find_bidirectional_paths(source, target, max_steps, weight_threshold, min_steps)
        if not candidate_paths:
            results.append({
                'source': source,
                'target': target,
                'path': "",
                'formatted_path': "No valid path found",
                'weights': "",
                'steps': 0
            })
        else:
            for path, weights in candidate_paths:
                formatted = ""
                for i, neuron in enumerate(path):
                    if i == 0:
                        formatted += str(neuron)
                    else:
                        formatted += " ->(" + str(weights[i-1]) + ")-> " + str(neuron)
                results.append({
                    'source': source,
                    'target': target,
                    'path': str(path),
                    'formatted_path': formatted,
                    'weights': str(weights),
                    'steps': len(path) - 1
                })
    return results

In [9]:
def save_automation_results_to_csv(results, file_name='automated_bidirectional_paths.csv'):
    """
    Save the automation results to a CSV file.
    
    Each row will contain:
      - 'source': the source neuron ID (prefixed with a ' to preserve formatting in Excel).
      - 'target': the target neuron ID (prefixed with a ' to preserve formatting in Excel).
      - 'path': the raw list (as a string) of neuron IDs from source to target.
      - 'formatted_path': a human-readable string representing the pathway.
      - 'weights': the list (as a string) of connection weights.
      - 'steps': the number of edges (steps) in the path.
    """
    df = pd.DataFrame(results)
    # Prepend a single quote (') to preserve the long numeric strings in Excel.
    df['source'] = "'" + df['source'].astype(str)
    df['target'] = "'" + df['target'].astype(str)
    
    df.to_csv(file_name, index=False)
    print(f"Automated candidate paths saved to {file_name}")

In [10]:
if __name__ == "__main__":
    # Example list of source neurons (your "A1" list)
    sources = [
        720575940623617973, #VP5 PN
        # Add more source neuron IDs as needed.
    ]
    
    # The target neuron (B1)
    target = 720575940634984800 #LNd6

    # Run the automated bidirectional search with progress tracking.
    results = automate_bidirectional_paths(sources, target, max_steps=6, weight_threshold=5, min_steps=2)
    
    # Save the results to a CSV file.
    save_automation_results_to_csv(results, file_name='automated_bidirectional_paths.csv')
    
    # Optionally, print the results.
    for row in results:
        print(row)

Processing sources:   0%|                                 | 0/1 [00:00<?, ?it/s]

Using materialization version 1002.
Using materialization version 1002.
Using materialization version 1002.


Fetching connectivity:   0%|          | 0/2 [00:00<?, ?it/s]

Processing sources: 100%|█████████████████████████| 1/1 [00:06<00:00,  6.62s/it]

Automated candidate paths saved to automated_bidirectional_paths.csv
{'source': 720575940623617973, 'target': 720575940634984800, 'path': '[720575940623617973, 720575940631997767, 720575940624899767, 720575940634984800]', 'formatted_path': '720575940623617973 ->(27)-> 720575940631997767 ->(7)-> 720575940624899767 ->(41)-> 720575940634984800', 'weights': '[27, 7, 41]', 'steps': 3}
{'source': 720575940623617973, 'target': 720575940634984800, 'path': '[720575940623617973, 720575940622658317, 720575940624899767, 720575940634984800]', 'formatted_path': '720575940623617973 ->(48)-> 720575940622658317 ->(13)-> 720575940624899767 ->(41)-> 720575940634984800', 'weights': '[48, 13, 41]', 'steps': 3}
{'source': 720575940623617973, 'target': 720575940634984800, 'path': '[720575940623617973, 720575940622658317, 720575940644609440, 720575940634984800]', 'formatted_path': '720575940623617973 ->(48)-> 720575940622658317 ->(5)-> 720575940644609440 ->(29)-> 720575940634984800', 'weights': '[48, 5, 29]',